In [4]:
!pip install pandas
!pip install bs4
!pip install requests


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


^C



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import json

In [2]:
product_information = []
product_skus = []
product_prices = []
product_ratings = []


In [3]:
base_url = 'https://www.microcenter.com'
for page_number in range(1, 18):
    url = f'{base_url}/search/search_results.aspx?N=4294967288&NTK=all&page={page_number}&cat=Laptops/Notebooks-:-MicroCenter'
    page = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(page.content, 'html.parser')
    #print(soup.prettify())
    #Pull ul out of product grid
    product_grid = soup.find('article', id='productGrid')
    #Pull li out of product grid
    product_list = product_grid.find_all('li', class_='product_wrapper')
    print(f"{len(product_list)} results on page {page_number}")
    #print(product_list)

    sku_regex = re.compile(r'(\d+)')

    price_regex = re.compile(r'([1-9,]*\d+\.\d+)')

    for product in product_list:
        #Get data ID from product, different from SKU used as unique identifier
        data_id = product.find('a', class_='image2 productClickItemV2').get('data-id')

        #print(product)
        # print(product)
        sku = product.find('p', class_='sku').text
        sku_match = sku_regex.search(sku)
        product_match = sku_match.group(1)
        product_skus.append(product_match)

        price = product.find('span', itemprop="price").text
        price_match = price_regex.search(price)
        product_prices.append(price_match.group(1))

        description = product.find('a', class_=f'productClickItemV2 ProductLink_{data_id}').text
        product_information.append(description)

        formatted_data_id = f"{int(data_id):07d}"

        redirect_path = product.find('div', {'data-bv-product-id': str(formatted_data_id)})

        redirect_url = base_url + redirect_path.get('data-bv-redirect-url')

        redirect_page = requests.get(redirect_url, headers={'User-Agent': 'Mozilla/5.0'})
        redirect_soup = BeautifulSoup(redirect_page.content, 'html.parser')

        #print(redirect_soup.prettify())

        #Pull review ratings from redirect page

        script_tag = redirect_soup.find_all('script', type='application/ld+json')
        print(script_tag)
        review_script_tag = None
        for script in script_tag:
            if "aggregateRating" in script.string:
                review_script_tag = script
                break

        if review_script_tag:
            json_data = json.loads(review_script_tag.string)
        
        # Step 5: Extract the aggregateRating data
        if "aggregateRating" in json_data:
            aggregate_rating = json_data["aggregateRating"]
            rating_value = aggregate_rating.get("ratingValue")
            review_count = aggregate_rating.get("reviewCount")

        product_ratings.append((rating_value, review_count))

        print(f"Product Prices: {product_prices}")
        print(f"Product SKUs: {product_skus}")
        print(f"Product Information: {product_information}")
        print(f"Product Ratings: {product_ratings}")

        # stock = product.find('div', class_='stock')
        # print(stock)
        # product_ratings.append(product.find('div', class_='bv_text').text)
        # product_information.append(product.find('a', class_='productCickItemV2').text)

        print(product_skus + product_prices)
        break
    
df = pd.DataFrame({'SKU': product_skus, 'Price': product_prices, 'Rating': product_ratings, 'Product Information': product_information})
df.to_csv('microcenter_laptops.csv', index=False)
print(df)



24 results on page 1
[<script type="application/ld+json">
{
"@context": "https://schema.org/", 
"@type": "BreadcrumbList", 
"itemListElement": [
{
"@type": "ListItem", 
"position": 1, 
"name": "Home",
"item": "https://www.microcenter.com"  
}
,{
"@type": "ListItem", 
"position": 2, 
"name": "Computers",
"item": "https://www.microcenter.com/category/4294964325/computers"  
}
,{
"@type": "ListItem", 
"position": 3, 
"name": "Laptops/Notebooks",
"item": "https://www.microcenter.com/category/4294967291/laptops-notebooks"  
}
,{
"@type": "ListItem", 
"position": 4, 
"name": "Laptops/Notebooks",
"item": "https://www.microcenter.com/category/4294967288/laptops-notebooks"  
}
]}
</script>, <script type="application/ld+json"> 
{
  "@context": "http://www.schema.org",
  "@type": "Organization",
  "name": "Micro Center",
  "url": "https://www.microcenter.com/",
  "logo": "https://90a1c75758623581b3f8-5c119c3de181c9857fcb2784776b17ef.ssl.cf2.rackcdn.com/images_sf_MC_logo.png",
    "contactPoint": 